<a href="https://colab.research.google.com/github/Linda-lls/GEOG0105/blob/main/5_SegFormer_proportion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Calculate the proportions of selected labels by SegFormer**

In [ ]:
!pip install transformers

In [ ]:
# Import packages
import torch
import os
import requests
import pandas as pd
import numpy as np
from transformers import AutoImageProcessor, SegformerForSemanticSegmentation
from PIL import Image
import requests

In [ ]:
# Load pretrained extractor and model into TensorFlow based implementation
image_processor = AutoImageProcessor.from_pretrained("nvidia/segformer-b5-finetuned-ade-640-640")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/segformer-b5-finetuned-ade-640-640")

Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
/usr/local/lib/python3.10/dist-packages/transformers/models/segformer/image_processing_segformer.py:99: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


In [ ]:
# Load sample images csv file
file_id = '1-0-O8KVBNlCDCEi1CMpTp14owyveHQ60'
csv_url = f'https://drive.google.com/uc?id={file_id}'
sample_images = pd.read_csv(csv_url)

sample_images

,filename_x,id_x,lon,lat,bearing,geometry,index_right,OBJECTID,PROVINCE,CAT_B,WARDNO,WARD_ID,SUM_WARD_P,SHAPE_LENG,WARD_EXT,ALBERS_ARE,SHAPE_Le_1,SHAPE_Area
0,0_angle_0.jpg.jpg,0,27.867442,-26.491770,160.6,POINT (27.86744182362429 -26.49176953583611),1392,1306,Gauteng,JHB,1,79800001,16406.0,0.157948,JHB_Ward_2011,7.425474,0.157948,0.000672
1,0_angle_1.jpg.jpg,0,27.867442,-26.491770,160.6,POINT (27.86744182362429 -26.49176953583611),1392,1306,Gauteng,JHB,1,79800001,16406.0,0.157948,JHB_Ward_2011,7.425474,0.157948,0.000672
2,0_angle_2.jpg.jpg,0,27.867442,-26.491770,160.6,POINT (27.86744182362429 -26.49176953583611),1392,1306,Gauteng,JHB,1,79800001,16406.0,0.157948,JHB_Ward_2011,7.425474,0.157948,0.000672
3,1_angle_0.jpg.jpg,1,27.856444,-26.484489,345.0,POINT (27.856443565981824 -26.484488536755784),1392,1306,Gauteng,JHB,1,79800001,16406.0,0.157948,JHB_Ward_2011,7.425474,0.157948,0.000672
4,1_angle_1.jpg.jpg,1,27.856444,-26.484489,345.0,POINT (27.856443565981824 -26.484488536755784),1392,1306,Gauteng,JHB,1,79800001,16406.0,0.157948,JHB_Ward_2011,7.425474,0.157948,0.000672
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15595,5198_angle_1.jpg.jpg,5198,27.815509,-26.247689,179.3,POINT (27.8155086 -26.247689049999998),2009,1435,Gauteng,JHB,130,79800130,13441.0,0.063921,JHB_Ward_2011,1.694833,0.063921,0.000153
15596,5198_angle_2.jpg.jpg,5198,27.815509,-26.247689,179.3,POINT (27.8155086 -26.247689049999998),2009,1435,Gauteng,JHB,130,79800130,13441.0,0.063921,JHB_Ward_2011,1.694833,0.063921,0.000153
15597,5199_angle_0.jpg.jpg,5199,27.819634,-26.253996,224.6,POINT (27.81963414528996 -26.25399588596665),2009,1435,Gauteng,JHB,130,79800130,13441.0,0.063921,JHB_Ward_2011,1.694833,0.063921,0.000153
15598,5199_angle_1.jpg.jpg,5199,27.819634,-26.253996,224.6,POINT (27.81963414528996 -26.25399588596665),2009,1435,Gauteng,JHB,130,79800130,13441.0,0.063921,JHB_Ward_2011,1.694833,0.063921,0.000153


In [ ]:
# Load label-to-index mapping csv file
# Load sample images csv file
file_id = '1wM5HJpQKnUVdAuJ7fqMT_PJ4NoGkpScR'
csv_url = f'https://drive.google.com/uc?id={file_id}'
label_mapping = pd.read_csv(csv_url)

label_mapping = label_mapping.drop(columns=["Idx"])
label_mapping

,Ratio,Train,Val,Stuff,Name
0,0.1576,11664,1172,1,wall
1,0.1072,6046,612,1,building;edifice
2,0.0878,8265,796,1,sky
3,0.0621,9336,917,1,floor;flooring
4,0.0480,6678,641,0,tree
...,...,...,...,...,...
145,0.0003,130,14,0,shower
146,0.0003,178,28,0,radiator
147,0.0002,504,57,0,glass;drinking;glass
148,0.0002,775,96,0,clock


In [ ]:
# Calculate the proportion of pixels for a specific class in an image
def calculate_proportion(image, label_id_to_calculate):
    inputs = image_processor(images=image, return_tensors="pt")
    outputs = model(**inputs)
    logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)
    # Convert logits to predicted class probabilities using softmax
    probs = torch.softmax(logits, dim=1)
    # Get the predicted class labels for each pixel (argmax along the channel dimension)
    predicted_labels = torch.argmax(probs, dim=1).squeeze()

    mask = predicted_labels == label_id_to_calculate
    num_pixels_with_label = mask.sum().item()
    total_pixels = predicted_labels.numel()
    proportion = num_pixels_with_label / total_pixels
    return proportion

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Iterate through images in the folder

# Create a list to store new rows with calculated proportions
new_rows = []

# Process each row in sample_images_info for the specified range
for index, row in sample_images.iloc[0:15600].iterrows():
    # Construct the image URL
    image_filename = row['filename_x']
    image_path = os.path.join("/content/drive/MyDrive/GEOG0105/GSV_final", image_filename)

    # Load the image
    image = Image.open(image_path)

    # Calculate proportions for each class (adjust label_ids as needed)
    label_ids_to_calculate = [1, 2, 4, 6, 11, 12, 20, 32, 43, 72, 93]

    proportions_for_image = []
    for label_id in label_ids_to_calculate:
        proportion = calculate_proportion(image, label_id)
        proportions_for_image.append(proportion)

    # Create a dictionary with the row data and calculated proportions
    new_row_data = row.to_dict()
    for i, label_id in enumerate(label_ids_to_calculate):
        new_row_data[str(label_id)] = proportions_for_image[i]

    # Append the new row to the list
    new_rows.append(new_row_data)

# Create a new DataFrame from the list of new rows
sample_images_segformer_proportions = pd.DataFrame(new_rows)

# Save the updated DataFrame with calculated proportions
sample_images_segformer_proportions.to_csv('/content/drive/MyDrive/GEOG0105/Output/segformer_reuslt.csv', index=False)